In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, utils

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import preprocessing
from PIL import Image
import time
import os
import copy
from collections import Counter

In [2]:
tabular_data = "../../data/SkidSteer_2019-08.csv"

df = pd.read_csv(tabular_data, index_col=1)
df['Unique_ID'] = df[['Source','item#']].apply(lambda x: '_'.join(x),axis = 1)
df = df.filter(['Unique_ID','Winning Bid','Hours Final','Age at Sale (bin)','Bucket','Engine','Tires','Transmission'], axis = 1)
df = df.rename(columns={
    'Unique_ID': "unique_id",
    'Hours Final': "hours_final",
    'Winning Bid': "winning_bid",
    'Age at Sale (bin)': "age_at_sale",
    'Bucket': "bucket",
    'Engine': "engine",
    'Tires': "tires",
    'Transmission': "transmission"
})
# color = pd.read_csv('skid_steer_color_score.csv')
# final_df = pd.merge(new_df, color,on='Unique_ID',how='inner')


### removal
# remove duplicant
duplicated_item = [item for item, count in Counter(df["unique_id"]).items() if count > 1]
df = df[~df['unique_id'].isin(duplicated_item)]

# remove not matched rows
image_item = [img_name.strip(".jpg") for img_name in os.listdir("../../data/images/")]
df = df[df["unique_id"].isin(image_item)]

# remove comma
df["winning_bid"] = df["winning_bid"].str.replace(',', '').astype(int)

# remove special image
df = df[df['unique_id'] != "rbauction_10525632"]

In [3]:
### winning_bid

# log-transform
df["winning_bid"] = np.log(df["winning_bid"])

# min max scale
mm_scaler_price = preprocessing.MinMaxScaler((-1, 1))
df["winning_bid"] = mm_scaler_price.fit_transform(df["winning_bid"].to_numpy().reshape(-1, 1))

In [4]:
### hours_final

# impute nan with median and new binary indicator
df["hours_final"] = df["hours_final"].str.replace(",", "")
df["hours_final"] = df["hours_final"].astype(float)
df.insert(3, column="hours_final_nan", value=df["hours_final"].isna().astype(int))
df.loc[df["hours_final"].isna(), "hours_final"] = df["hours_final"].median(skipna=True)

# log transform
df["hours_final"] = np.log(df["hours_final"])

# normalize
rb_scaler_hour = preprocessing.RobustScaler()
df["hours_final"] = rb_scaler_hour.fit_transform(np.array(df["hours_final"]).reshape(-1, 1))

In [5]:
### age_at_sale

# impute nan with median and new binary indicator
df["age_at_sale"] = df["age_at_sale"].astype(float)
df.insert(5, column="age_at_sale_nan", value=df["age_at_sale"].isna().astype(int))
df.loc[df["age_at_sale"].isna(), "age_at_sale"] = df["age_at_sale"].median(skipna=True)

# normalize
rb_scaler_age = preprocessing.RobustScaler()
df["age_at_sale"] = rb_scaler_age.fit_transform(np.array(df["age_at_sale"]).reshape(-1, 1))

In [6]:
### bucket
df.insert(7, column="bucket_bin", value=0)
df.loc[
    ~df["bucket"].isna() & 
    df["bucket"].str.contains("bucket", case=False) | 
    df["bucket"].str.contains("bkt", case=False), "bucket_bin"
] = 1

In [7]:
# shuffle and split
np.random.seed(1)
split = [0.7, 0.3]
split0 = round(df.shape[0] * split[0])
# split1 = round(df.shape[0] * (split[0] + split[1]))
df = df.sample(frac=1)
df_train = df.iloc[:split0]
df_val = df.iloc[split0:]

In [8]:
df_train.head()

,unique_id,winning_bid,hours_final,hours_final_nan,age_at_sale,age_at_sale_nan,bucket,bucket_bin,engine,tires,transmission
Item Id,,,,,,,,,,,
ec1907ce-5c12-5ab8-b42d-39eb481e6049,ironplanet_1703726,0.105406,-6.732824,0,-0.666667,0,"74"" Wide General Purpose Smooth Edge Bucket",1,NaN,Cushion Tires,NaN
529071,PW_DD1289,0.172815,-1.629216,0,-1.000000,0,"Kubota 68""W bucket",1,Kubota V3307-CR four cylinder turbo diesel engine,Titan 12-16.5 NHS tires,Two speed hydrostatic transmission
eff85cf8-4988-8990-b317-39e91a64ec3a,rbauction_10239624,0.219707,-0.164164,0,-0.666667,0,hyd Q/C bkt,1,NaN,NaN,NaN
386158,PW_H1380,-0.072566,-0.769452,0,-0.333333,0,"80""W bucket",1,"81 HP, Case 3.2L four cylinder turbo diesel en...",12-16.5 tires,Hydrostatic transmission
460813,PW_J8873,-0.305592,-0.416323,0,-0.333333,0,NaN,0,John Deere 5030TT001 3.0L turbo diesel engine,12-16.5 tires,Two speed hydrostatic transmission


In [9]:
df_val.head()

,unique_id,winning_bid,hours_final,hours_final_nan,age_at_sale,age_at_sale_nan,bucket,bucket_bin,engine,tires,transmission
Item Id,,,,,,,,,,,
cdf26f3a-f792-7a0f-350e-39e9461c54c9,bigiron_EN9531,0.164709,-0.503074,0,-0.333333,0,"84"" Bucket",1,"4-Cyl Turbo Diesel Engine, 84 Hp","Some Tires Have Cuts (Pictured), Tires- 14-17....",NaN
14c666d4-512a-8962-6072-39e918e5b2f7,rbauction_10693873,-0.861828,-0.443163,0,1.333333,0,bkt,1,NaN,NaN,NaN
dc20ac44-7c35-11a7-2dfa-39e8cc01c274,bigiron_BO0203,-0.180158,-0.226316,0,1.333333,0,"Buckets 12"", 64"" Bucket",1,58 Horse Power Isuzu Diesel Engine,"12-16.5 Tires, Spare Tire And Rim",Hydrostat Transmission
ebfb5935-40e0-4488-2e6c-39ea98658096,ironplanet_1891010,-0.402470,-6.732824,0,-0.333333,0,"72"" Wide General Purpose Smooth Edge Bucket",1,NaN,NaN,NaN
446849,PW_J2936,-0.402470,-1.926828,0,-0.333333,0,NaN,0,"Case four cylinder turbo diesel engine, Non-op...",12-16.5 tires,Hydrostatic transmission


In [10]:
df_train.to_csv("./SkidSteer_2019-08_clean_train.csv")
df_val.to_csv("./SkidSteer_2019-08_clean_val.csv")

In [11]:
class skidsteer_dataset(Dataset):
    """Corrosion Detection dataset."""

    def __init__(self, 
                 csv_file, 
                 img_root, 
                 transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            img_root (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample (including augmentation).
        """
        self.csv_file = pd.read_csv(csv_file, index_col=0)
        self.img_root = img_root
        self.transform = transform

    def __len__(self):
        return len(self.csv_file)

    def __getitem__(self, idx):
        '''Return one data point with a PIL image and its label.'''
        img_dir = os.path.join(self.img_root, self.csv_file["unique_id"][idx]) + ".jpg"
        price = self.csv_file["winning_bid"][idx]
        image = Image.open(img_dir)
        others = torch.tensor(self.csv_file.iloc[idx, [2, 3, 4, 5, 7]])
        
        if self.transform:
            image = self.transform(image)
        sample = {'image': image, 'price': price, "others": others}
        return sample

In [12]:
def norm2price(tensor, min_max_scaler):
    array2d = tensor.to("cpu").data.numpy().reshape(-1, 1)
    return np.exp(min_max_scaler.inverse_transform(array2d))

def price_MAE(outputs, prices, min_max_scaler):
    outputs = norm2price(outputs, min_max_scaler)
    prices = norm2price(prices, min_max_scaler)
    return np.abs(outputs - prices).mean()

In [13]:
def train_model(model, model_LU, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch = None
    best_loss = float("Inf")
    best_mae = float("Inf")

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            running_loss = 0.0
            running_mae = 0.0
            
            # Iterate over data.
            for items in dataloaders[phase]:
                images = items["image"].to(device)
                prices = items["price"].to(device)
                others = items["others"].to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = cat_net(model, model_LU, images, others).squeeze()
                    loss = criterion(outputs, prices)
                    mae = price_MAE(outputs, prices, mm_scaler_price)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * images.size(0)
                running_mae += mae * images.size(0)
            if phase == 'train':
                scheduler.step()
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_mae = running_mae / dataset_sizes[phase]
            print('{} Loss: {:.4f}'.format(phase, epoch_loss))
            print('{} MAE: {:.4f}'.format(phase, epoch_mae))
            
            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                best_epoch = epoch + 1
                best_loss = epoch_loss
                best_mae = epoch_mae
                best_model_wts = copy.deepcopy(model.state_dict())
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Loss: {:4f} at epoch {}'.format(best_loss, best_epoch))
    print('Best val MAE: {:4f} at epoch {}'.format(best_mae, best_epoch))

    # load best model weights
    print("\nLoad the model weights at the best epoch")
    model.load_state_dict(best_model_wts)
    return model

In [14]:
CSV_FILE = {"train": "./SkidSteer_2019-08_clean_train.csv",
            "val": "./SkidSteer_2019-08_clean_val.csv"}
IMG_ROOT = "../../data/images/"
TRANSFORM = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

datasets = {x: skidsteer_dataset(csv_file=CSV_FILE[x],
                                 img_root=IMG_ROOT,
                                 transform=TRANSFORM[x])
            for x in ["train", "val"]}
dataloaders = {x: DataLoader(datasets[x], 
                             batch_size=16, 
                             shuffle=True, 
                             num_workers=4)
               for x in ["train", "val"]}
dataset_sizes = {x: len(datasets[x]) for x in ["train", "val"]}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [15]:
def cat_net(model, model_LU, images, others):
    z = torch.cat((model(images), others), dim=1)
    return model_LU(z)

## ResNet152

- Structure
    - (2053, 1)
    - (2053, 32, 1)
    - (2053, 64, 1)
    - (2053, 128, 1)
    - (2053, 256, 1)
    - (2053, 256, 32, 1)
    - (2053, 256, 64, 1)
- DO
    - after structure was decided
- FineTuning
    - Last CB
    - Last 2 CB
- Loss function 
    - MAE
    - HUB

### 1. FT last CB

In [16]:
torch.manual_seed(0)

### ResNet18
model_ft = models.resnet152(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Identity()

### Freeze and Fine Tuning
for param in model_ft.parameters():
    param.requires_grad = False
for param in model_ft.layer4.parameters():
    param.requires_grad = True
assert [param.requires_grad for param in model_ft.parameters()][-1]

### LU
model_LU = nn.Sequential(
    nn.Linear(num_ftrs + 5, 1), 
    nn.Tanh()
)

### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(list(model_ft.parameters())+list(model_LU.parameters()), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [17]:
model_res152 = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
train Loss: 0.0946
train MAE: 4632.0084
val Loss: 0.0755
val MAE: 3984.7289

Epoch 2/50
----------
train Loss: 0.0742
train MAE: 4137.6157
val Loss: 0.0905
val MAE: 5128.0544

Epoch 3/50
----------
train Loss: 0.0627
train MAE: 3770.1253
val Loss: 0.0534
val MAE: 3308.9413

Epoch 4/50
----------
train Loss: 0.0627
train MAE: 3798.0078
val Loss: 0.0448
val MAE: 3130.7558

Epoch 5/50
----------
train Loss: 0.0635
train MAE: 3830.6548
val Loss: 0.0425
val MAE: 2959.7879

Epoch 6/50
----------
train Loss: 0.0595
train MAE: 3693.2396
val Loss: 0.0589
val MAE: 3547.8486

Epoch 7/50
----------
train Loss: 0.0568
train MAE: 3615.4677
val Loss: 0.0547
val MAE: 3325.9065

Epoch 8/50
----------
train Loss: 0.0555
train MAE: 3593.1478
val Loss: 0.0490
val MAE: 3322.5282

Epoch 9/50
----------
train Loss: 0.0560
train MAE: 3667.6437
val Loss: 0.0675
val MAE: 4343.6445

Epoch 10/50
----------
train Loss: 0.0518
train MAE: 3441.4072
val Loss: 0.0441
val MAE: 3124.5430

Epoch 11/

### 1.1. FT last 2 CBs

In [18]:
torch.manual_seed(0)

### ResNet18
model_ft = models.resnet152(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Identity()

### Freeze and Fine Tuning
for param in model_ft.parameters():
    param.requires_grad = False
for param in model_ft.layer3.parameters():
    param.requires_grad = True
for param in model_ft.layer4.parameters():
    param.requires_grad = True
assert [param.requires_grad for param in model_ft.parameters()][-1]

### LU
model_LU = nn.Sequential(
    nn.Linear(num_ftrs + 5, 1), 
    nn.Tanh()
)

### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(list(model_ft.parameters())+list(model_LU.parameters()), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [19]:
model_res152 = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
train Loss: 0.0925
train MAE: 4583.8335
val Loss: 0.0680
val MAE: 3771.5593

Epoch 2/50
----------
train Loss: 0.0712
train MAE: 4046.2355
val Loss: 0.0868
val MAE: 5059.0461

Epoch 3/50
----------
train Loss: 0.0599
train MAE: 3665.4739
val Loss: 0.0454
val MAE: 3081.9575

Epoch 4/50
----------
train Loss: 0.0579
train MAE: 3628.0286
val Loss: 0.0416
val MAE: 2941.0531

Epoch 5/50
----------
train Loss: 0.0582
train MAE: 3652.9002
val Loss: 0.0389
val MAE: 2853.1698

Epoch 6/50
----------
train Loss: 0.0581
train MAE: 3643.9536
val Loss: 0.0574
val MAE: 3464.8634

Epoch 7/50
----------
train Loss: 0.0571
train MAE: 3607.6350
val Loss: 0.0522
val MAE: 3232.9503

Epoch 8/50
----------
train Loss: 0.0533
train MAE: 3526.5299
val Loss: 0.0407
val MAE: 2944.5676

Epoch 9/50
----------
train Loss: 0.0499
train MAE: 3461.0181
val Loss: 0.0570
val MAE: 3783.3900

Epoch 10/50
----------
train Loss: 0.0455
train MAE: 3196.2961
val Loss: 0.0387
val MAE: 2805.5764

Epoch 11/

### 2. new Baseline - no activ

In [20]:
torch.manual_seed(0)

### ResNet18
model_ft = models.resnet152(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Identity()

### Freeze and Fine Tuning
# for param in model_ft.parameters():
#     param.requires_grad = False
# for param in model_ft.layer4.parameters():
#     param.requires_grad = True
# assert [param.requires_grad for param in model_ft.parameters()][-1]

### LU
model_LU = nn.Sequential(
    nn.Linear(num_ftrs + 5, 1)
)

### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(list(model_ft.parameters())+list(model_LU.parameters()), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [21]:
model_res152 = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
train Loss: 0.1015
train MAE: 4805.1882
val Loss: 0.0556
val MAE: 3403.8458

Epoch 2/50
----------
train Loss: 0.0705
train MAE: 3967.9539
val Loss: 0.0543
val MAE: 3456.8726

Epoch 3/50
----------
train Loss: 0.0876
train MAE: 4516.0662
val Loss: 0.2840
val MAE: 12286.2214

Epoch 4/50
----------
train Loss: 0.0681
train MAE: 3937.5331
val Loss: 0.0529
val MAE: 3446.8097

Epoch 5/50
----------
train Loss: 0.0774
train MAE: 4206.6847
val Loss: 0.0512
val MAE: 3301.1513

Epoch 6/50
----------
train Loss: 0.0602
train MAE: 3698.9219
val Loss: 0.0421
val MAE: 2917.4578

Epoch 7/50
----------
train Loss: 0.0587
train MAE: 3661.2071
val Loss: 0.0448
val MAE: 2986.3350

Epoch 8/50
----------
train Loss: 0.0617
train MAE: 3763.3796
val Loss: 0.0387
val MAE: 2821.1911

Epoch 9/50
----------
train Loss: 0.0655
train MAE: 3911.3783
val Loss: 0.0561
val MAE: 3710.3154

Epoch 10/50
----------
train Loss: 0.0537
train MAE: 3473.4281
val Loss: 0.0583
val MAE: 3670.5780

Epoch 11

### 2.1. new Baseline - no activ + MAE

In [22]:
torch.manual_seed(0)

### ResNet18
model_ft = models.resnet152(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Identity()

### Freeze and Fine Tuning
# for param in model_ft.parameters():
#     param.requires_grad = False
# for param in model_ft.layer4.parameters():
#     param.requires_grad = True
# assert [param.requires_grad for param in model_ft.parameters()][-1]

### LU
model_LU = nn.Sequential(
    nn.Linear(num_ftrs + 5, 1)
)

### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.L1Loss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(list(model_ft.parameters())+list(model_LU.parameters()), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [23]:
model_res152 = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
train Loss: 0.3041
train MAE: 5892.9842
val Loss: 0.2451
val MAE: 4853.5876

Epoch 2/50
----------
train Loss: 0.2743
train MAE: 5220.7951
val Loss: 0.1855
val MAE: 3375.8071

Epoch 3/50
----------
train Loss: 0.2889
train MAE: 5483.2238
val Loss: 0.5029
val MAE: 13236.8785

Epoch 4/50
----------
train Loss: 0.2563
train MAE: 4916.5033
val Loss: 0.3959
val MAE: 9303.2580

Epoch 5/50
----------
train Loss: 0.2385
train MAE: 4528.9640
val Loss: 0.2733
val MAE: 4547.5367

Epoch 6/50
----------
train Loss: 0.2387
train MAE: 4526.9927
val Loss: 0.1824
val MAE: 3355.6091

Epoch 7/50
----------
train Loss: 0.2132
train MAE: 4034.5163
val Loss: 0.1816
val MAE: 3235.2384

Epoch 8/50
----------
train Loss: 0.1980
train MAE: 3771.5298
val Loss: 0.2118
val MAE: 4031.4774

Epoch 9/50
----------
train Loss: 0.1870
train MAE: 3604.4312
val Loss: 0.2025
val MAE: 3846.0696

Epoch 10/50
----------
train Loss: 0.1850
train MAE: 3514.3448
val Loss: 0.1674
val MAE: 3011.3945

Epoch 11

### 2.2. new Baseline - no activ + HUB

In [24]:
torch.manual_seed(0)

### ResNet18
model_ft = models.resnet152(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Identity()

### Freeze and Fine Tuning
# for param in model_ft.parameters():
#     param.requires_grad = False
# for param in model_ft.layer4.parameters():
#     param.requires_grad = True
# assert [param.requires_grad for param in model_ft.parameters()][-1]

### LU
model_LU = nn.Sequential(
    nn.Linear(num_ftrs + 5, 1)
)

### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.SmoothL1Loss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(list(model_ft.parameters())+list(model_LU.parameters()), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [25]:
model_res152 = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
train Loss: 0.0453
train MAE: 4473.1590
val Loss: 0.0308
val MAE: 3580.1740

Epoch 2/50
----------
train Loss: 0.0347
train MAE: 3988.6413
val Loss: 0.0281
val MAE: 3495.3021

Epoch 3/50
----------
train Loss: 0.0295
train MAE: 3624.6763
val Loss: 0.0228
val MAE: 3084.8192

Epoch 4/50
----------
train Loss: 0.0268
train MAE: 3485.6983
val Loss: 0.0230
val MAE: 3121.7730

Epoch 5/50
----------
train Loss: 0.0284
train MAE: 3583.6530
val Loss: 0.0205
val MAE: 2912.6241

Epoch 6/50
----------
train Loss: 0.0325
train MAE: 3807.8303
val Loss: 0.0217
val MAE: 3017.6306

Epoch 7/50
----------
train Loss: 0.0295
train MAE: 3646.7202
val Loss: 0.0255
val MAE: 3168.0305

Epoch 8/50
----------
train Loss: 0.0272
train MAE: 3552.1003
val Loss: 0.0210
val MAE: 2944.9678

Epoch 9/50
----------
train Loss: 0.0242
train MAE: 3360.1028
val Loss: 0.0387
val MAE: 4642.1931

Epoch 10/50
----------
train Loss: 0.0242
train MAE: 3284.3950
val Loss: 0.0276
val MAE: 3296.7845

Epoch 11/

### 2.3. new Baseline - no activ + Adam

In [16]:
torch.manual_seed(0)

### ResNet18
model_ft = models.resnet152(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Identity()

### Freeze and Fine Tuning
# for param in model_ft.parameters():
#     param.requires_grad = False
# for param in model_ft.layer4.parameters():
#     param.requires_grad = True
# assert [param.requires_grad for param in model_ft.parameters()][-1]

### LU
model_LU = nn.Sequential(
    nn.Linear(num_ftrs + 5, 1)
)

### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(list(model_ft.parameters())+list(model_LU.parameters()))
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [17]:
model_res152 = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
train Loss: 0.1432
train MAE: 5740.3212
val Loss: 0.9615
val MAE: 74626.6057

Epoch 2/50
----------
train Loss: 0.0747
train MAE: 4094.4628
val Loss: 0.0969
val MAE: 4986.5934

Epoch 3/50
----------
train Loss: 0.0650
train MAE: 3829.5950
val Loss: 0.0675
val MAE: 3809.2370

Epoch 4/50
----------
train Loss: 0.0740
train MAE: 4117.0302
val Loss: 0.0803
val MAE: 4230.1910

Epoch 5/50
----------
train Loss: 0.0690
train MAE: 3992.8403
val Loss: 0.0786
val MAE: 4214.6596

Epoch 6/50
----------
train Loss: 0.0658
train MAE: 3912.0867
val Loss: 0.0653
val MAE: 3799.3333

Epoch 7/50
----------
train Loss: 0.0651
train MAE: 3884.5665
val Loss: 0.0633
val MAE: 3720.9165

Epoch 8/50
----------
train Loss: 0.0644
train MAE: 3844.1338
val Loss: 0.0632
val MAE: 3749.3435

Epoch 9/50
----------
train Loss: 0.0612
train MAE: 3758.1480
val Loss: 0.0679
val MAE: 3804.9440

Epoch 10/50
----------
train Loss: 0.0605
train MAE: 3717.6837
val Loss: 0.0619
val MAE: 3571.9946

Epoch 11

### 2.2. new Baseline - no activ + Adadelta

In [18]:
torch.manual_seed(0)

### ResNet18
model_ft = models.resnet152(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Identity()

### Freeze and Fine Tuning
# for param in model_ft.parameters():
#     param.requires_grad = False
# for param in model_ft.layer4.parameters():
#     param.requires_grad = True
# assert [param.requires_grad for param in model_ft.parameters()][-1]

### LU
model_LU = nn.Sequential(
    nn.Linear(num_ftrs + 5, 1)
)

### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.Adadelta(list(model_ft.parameters())+list(model_LU.parameters()))
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [19]:
model_res152 = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
train Loss: 0.2442
train MAE: 1857348.7576
val Loss: 0.0735
val MAE: 3973.1637

Epoch 2/50
----------
train Loss: 0.0801
train MAE: 4296.7992
val Loss: 0.0710
val MAE: 3970.3748

Epoch 3/50
----------
train Loss: 0.0718
train MAE: 4067.4088
val Loss: 0.0750
val MAE: 4085.7726

Epoch 4/50
----------
train Loss: 0.0665
train MAE: 3921.9143
val Loss: 0.0708
val MAE: 3901.6633

Epoch 5/50
----------
train Loss: 0.0637
train MAE: 3834.5348
val Loss: 0.0656
val MAE: 3777.7454

Epoch 6/50
----------
train Loss: 0.0628
train MAE: 3808.9559
val Loss: 0.0694
val MAE: 3848.8757

Epoch 7/50
----------
train Loss: 0.0618
train MAE: 3767.0009
val Loss: 0.0617
val MAE: 3659.3492

Epoch 8/50
----------
train Loss: 0.0607
train MAE: 3723.7588
val Loss: 0.0613
val MAE: 3676.5392

Epoch 9/50
----------
train Loss: 0.0579
train MAE: 3652.5166
val Loss: 0.0573
val MAE: 3473.4846

Epoch 10/50
----------
train Loss: 0.0567
train MAE: 3593.0581
val Loss: 0.0549
val MAE: 3370.1776

Epoch 

### 3. (2053, 32, 1)

In [20]:
torch.manual_seed(0)

### ResNet18
model_ft = models.resnet152(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Identity()

### Freeze and Fine Tuning
for param in model_ft.parameters():
    param.requires_grad = False
for param in model_ft.layer4.parameters():
    param.requires_grad = True
assert [param.requires_grad for param in model_ft.parameters()][-1]

### LU
model_LU = nn.Sequential(
    nn.Linear(num_ftrs + 5, 32), 
    nn.Tanh(),
    nn.Linear(32, 1)
)

### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(list(model_ft.parameters())+list(model_LU.parameters()), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [21]:
model_res152 = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
train Loss: 0.0846
train MAE: 4384.4155
val Loss: 0.0528
val MAE: 3387.7838

Epoch 2/50
----------
train Loss: 0.0618
train MAE: 3756.1548
val Loss: 0.0473
val MAE: 3178.0174

Epoch 3/50
----------
train Loss: 0.0557
train MAE: 3551.9656
val Loss: 0.0458
val MAE: 3091.0639

Epoch 4/50
----------
train Loss: 0.0531
train MAE: 3447.3675
val Loss: 0.0431
val MAE: 3017.4347

Epoch 5/50
----------
train Loss: 0.0499
train MAE: 3326.5685
val Loss: 0.0468
val MAE: 3146.0869

Epoch 6/50
----------
train Loss: 0.0514
train MAE: 3383.5831
val Loss: 0.0509
val MAE: 3330.3301

Epoch 7/50
----------
train Loss: 0.0471
train MAE: 3261.7553
val Loss: 0.0398
val MAE: 2894.3265

Epoch 8/50
----------
train Loss: 0.0484
train MAE: 3264.3000
val Loss: 0.0449
val MAE: 3106.9199

Epoch 9/50
----------
train Loss: 0.0450
train MAE: 3162.9088
val Loss: 0.0389
val MAE: 2839.1677

Epoch 10/50
----------
train Loss: 0.0461
train MAE: 3205.4823
val Loss: 0.0394
val MAE: 2863.7250

Epoch 11/

### 4. (2053, 64, 1)

In [22]:
torch.manual_seed(0)

### ResNet18
model_ft = models.resnet152(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Identity()

### Freeze and Fine Tuning
for param in model_ft.parameters():
    param.requires_grad = False
for param in model_ft.layer4.parameters():
    param.requires_grad = True
assert [param.requires_grad for param in model_ft.parameters()][-1]

### LU
model_LU = nn.Sequential(
    nn.Linear(num_ftrs + 5, 64), 
    nn.Tanh(),
    nn.Linear(64, 1)
)

### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(list(model_ft.parameters())+list(model_LU.parameters()), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [23]:
model_res152 = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
train Loss: 0.0900
train MAE: 4426.4513
val Loss: 0.0546
val MAE: 3447.3386

Epoch 2/50
----------
train Loss: 0.0637
train MAE: 3797.4818
val Loss: 0.0526
val MAE: 3369.8310

Epoch 3/50
----------
train Loss: 0.0579
train MAE: 3627.5210
val Loss: 0.0514
val MAE: 3336.5664

Epoch 4/50
----------
train Loss: 0.0536
train MAE: 3474.6154
val Loss: 0.0496
val MAE: 3244.8165

Epoch 5/50
----------
train Loss: 0.0514
train MAE: 3417.4062
val Loss: 0.0435
val MAE: 3049.7896

Epoch 6/50
----------
train Loss: 0.0494
train MAE: 3323.8502
val Loss: 0.0471
val MAE: 3223.5966

Epoch 7/50
----------
train Loss: 0.0485
train MAE: 3334.5064
val Loss: 0.0434
val MAE: 3044.4677

Epoch 8/50
----------
train Loss: 0.0477
train MAE: 3269.7019
val Loss: 0.0395
val MAE: 2875.7593

Epoch 9/50
----------
train Loss: 0.0465
train MAE: 3228.6397
val Loss: 0.0387
val MAE: 2843.7776

Epoch 10/50
----------
train Loss: 0.0440
train MAE: 3132.8730
val Loss: 0.0499
val MAE: 3212.9754

Epoch 11/

### 5. (2053, 128, 1)

In [24]:
torch.manual_seed(0)

### ResNet18
model_ft = models.resnet152(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Identity()

### Freeze and Fine Tuning
for param in model_ft.parameters():
    param.requires_grad = False
for param in model_ft.layer4.parameters():
    param.requires_grad = True
assert [param.requires_grad for param in model_ft.parameters()][-1]

### LU
model_LU = nn.Sequential(
    nn.Linear(num_ftrs + 5, 128), 
    nn.Tanh(),
    nn.Linear(128, 1)
)

### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(list(model_ft.parameters())+list(model_LU.parameters()), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [25]:
model_res152 = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
train Loss: 0.0859
train MAE: 4365.1549
val Loss: 0.0553
val MAE: 3438.6790

Epoch 2/50
----------
train Loss: 0.0613
train MAE: 3730.1841
val Loss: 0.0743
val MAE: 4093.2725

Epoch 3/50
----------
train Loss: 0.0568
train MAE: 3572.8577
val Loss: 0.0443
val MAE: 3047.7276

Epoch 4/50
----------
train Loss: 0.0544
train MAE: 3490.8998
val Loss: 0.0447
val MAE: 3094.1614

Epoch 5/50
----------
train Loss: 0.0516
train MAE: 3388.9747
val Loss: 0.0431
val MAE: 2994.4035

Epoch 6/50
----------
train Loss: 0.0499
train MAE: 3360.5436
val Loss: 0.0422
val MAE: 2973.7267

Epoch 7/50
----------
train Loss: 0.0467
train MAE: 3249.3826
val Loss: 0.0480
val MAE: 3230.4544

Epoch 8/50
----------
train Loss: 0.0506
train MAE: 3379.2338
val Loss: 0.0401
val MAE: 2882.4653

Epoch 9/50
----------
train Loss: 0.0457
train MAE: 3219.9541
val Loss: 0.0426
val MAE: 2976.9276

Epoch 10/50
----------
train Loss: 0.0450
train MAE: 3181.1243
val Loss: 0.0461
val MAE: 3135.6026

Epoch 11/

### 6. (2053, 256, 1)

In [26]:
torch.manual_seed(0)

### ResNet18
model_ft = models.resnet152(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Identity()

### Freeze and Fine Tuning
for param in model_ft.parameters():
    param.requires_grad = False
for param in model_ft.layer4.parameters():
    param.requires_grad = True
assert [param.requires_grad for param in model_ft.parameters()][-1]

### LU
model_LU = nn.Sequential(
    nn.Linear(num_ftrs + 5, 256), 
    nn.Tanh(),
    nn.Linear(256, 1)
)

### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(list(model_ft.parameters())+list(model_LU.parameters()), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [27]:
model_res152 = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
train Loss: 0.0886
train MAE: 4398.1366
val Loss: 0.0586
val MAE: 3580.3642

Epoch 2/50
----------
train Loss: 0.0633
train MAE: 3766.2058
val Loss: 0.0538
val MAE: 3372.5422

Epoch 3/50
----------
train Loss: 0.0582
train MAE: 3597.6707
val Loss: 0.0448
val MAE: 3084.0205

Epoch 4/50
----------
train Loss: 0.0529
train MAE: 3493.4358
val Loss: 0.0444
val MAE: 3085.4242

Epoch 5/50
----------
train Loss: 0.0517
train MAE: 3421.1513
val Loss: 0.0414
val MAE: 2941.1445

Epoch 6/50
----------
train Loss: 0.0482
train MAE: 3301.0774
val Loss: 0.0441
val MAE: 3047.3382

Epoch 7/50
----------
train Loss: 0.0502
train MAE: 3368.0528
val Loss: 0.0514
val MAE: 3347.7707

Epoch 8/50
----------
train Loss: 0.0471
train MAE: 3231.8974
val Loss: 0.0390
val MAE: 2867.5613

Epoch 9/50
----------
train Loss: 0.0488
train MAE: 3300.7274
val Loss: 0.0421
val MAE: 2982.2638

Epoch 10/50
----------
train Loss: 0.0467
train MAE: 3225.7588
val Loss: 0.0390
val MAE: 2866.5122

Epoch 11/

### 7. (2053, 256, 32, 1)

In [28]:
torch.manual_seed(0)

### ResNet18
model_ft = models.resnet152(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Identity()

### Freeze and Fine Tuning
for param in model_ft.parameters():
    param.requires_grad = False
for param in model_ft.layer4.parameters():
    param.requires_grad = True
assert [param.requires_grad for param in model_ft.parameters()][-1]

### LU
model_LU = nn.Sequential(
    nn.Linear(num_ftrs + 5, 256), 
    nn.Tanh(),
    nn.Linear(256, 32),
    nn.Tanh(),
    nn.Linear(32, 1)
)

### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(list(model_ft.parameters())+list(model_LU.parameters()), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [29]:
model_res152 = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
train Loss: 0.0828
train MAE: 4316.6808
val Loss: 0.0704
val MAE: 3933.6702

Epoch 2/50
----------
train Loss: 0.0649
train MAE: 3828.6423
val Loss: 0.0606
val MAE: 3646.8897

Epoch 3/50
----------
train Loss: 0.0562
train MAE: 3514.2853
val Loss: 0.0458
val MAE: 3104.3574

Epoch 4/50
----------
train Loss: 0.0557
train MAE: 3564.2806
val Loss: 0.0451
val MAE: 3089.1746

Epoch 5/50
----------
train Loss: 0.0529
train MAE: 3436.3025
val Loss: 0.0431
val MAE: 3009.8860

Epoch 6/50
----------
train Loss: 0.0497
train MAE: 3330.7514
val Loss: 0.0527
val MAE: 3384.7782

Epoch 7/50
----------
train Loss: 0.0483
train MAE: 3297.7667
val Loss: 0.0407
val MAE: 2931.0723

Epoch 8/50
----------
train Loss: 0.0453
train MAE: 3218.9903
val Loss: 0.0407
val MAE: 2899.0892

Epoch 9/50
----------
train Loss: 0.0473
train MAE: 3268.8500
val Loss: 0.0425
val MAE: 2983.6723

Epoch 10/50
----------
train Loss: 0.0465
train MAE: 3232.2897
val Loss: 0.0393
val MAE: 2883.2003

Epoch 11/

### 8. (2053, 256, 64, 1)

In [30]:
torch.manual_seed(0)

### ResNet18
model_ft = models.resnet152(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Identity()

### Freeze and Fine Tuning
for param in model_ft.parameters():
    param.requires_grad = False
for param in model_ft.layer4.parameters():
    param.requires_grad = True
assert [param.requires_grad for param in model_ft.parameters()][-1]

### LU
model_LU = nn.Sequential(
    nn.Linear(num_ftrs + 5, 256), 
    nn.Tanh(),
    nn.Linear(256, 64),
    nn.Tanh(),
    nn.Linear(64, 1)
)
### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(list(model_ft.parameters())+list(model_LU.parameters()), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [31]:
model_res152 = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
train Loss: 0.0871
train MAE: 4422.7633
val Loss: 0.0562
val MAE: 3503.3261

Epoch 2/50
----------
train Loss: 0.0628
train MAE: 3750.8937
val Loss: 0.0526
val MAE: 3353.5125

Epoch 3/50
----------
train Loss: 0.0566
train MAE: 3590.0992
val Loss: 0.0500
val MAE: 3288.3966

Epoch 4/50
----------
train Loss: 0.0551
train MAE: 3522.1797
val Loss: 0.0433
val MAE: 3032.7232

Epoch 5/50
----------
train Loss: 0.0543
train MAE: 3475.2599
val Loss: 0.0439
val MAE: 3045.1477

Epoch 6/50
----------
train Loss: 0.0502
train MAE: 3381.3188
val Loss: 0.0405
val MAE: 2898.3870

Epoch 7/50
----------
train Loss: 0.0484
train MAE: 3284.0751
val Loss: 0.0426
val MAE: 3010.8691

Epoch 8/50
----------
train Loss: 0.0467
train MAE: 3231.4209
val Loss: 0.0683
val MAE: 4058.5045

Epoch 9/50
----------
train Loss: 0.0463
train MAE: 3242.6908
val Loss: 0.0439
val MAE: 3069.6968

Epoch 10/50
----------
train Loss: 0.0451
train MAE: 3177.1206
val Loss: 0.0410
val MAE: 2947.5755

Epoch 11/